In [7]:
import os
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [8]:


# Función para extraer el nombre del anime desde la URL
def extract_anime_name(url):
    # Eliminar la parte inicial de la URL hasta "/manga/"
    parts = url.split('/manga/')
    if len(parts) > 1:
        # Obtener la parte después de "/manga/"
        name = parts[1].split('/')[0]
        return name.replace('-', ' ').title()
    return None



def extract_segmento_url(url, x , i):
    # Eliminar la parte inicial de la URL hasta "/manga/"
    parts = url.split('/')
    if len(parts) > 1:
        
        return '/'.join(parts[:6])+'/'+x+'/'+i
    return None

In [9]:

import os
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


# Función para extraer el nombre del anime desde la URL
def extract_anime_name(url):
    # Eliminar la parte inicial de la URL hasta "/manga/"
    parts = url.split('/manga/')
    if len(parts) > 1:
        # Obtener la parte después de "/manga/"
        name = parts[1].split('/')[0]
        return name.replace('-', ' ').title()
    return None



def extract_segmento_url(url, x , i):
    # Eliminar la parte inicial de la URL hasta "/manga/"
    parts = url.split('/')
    if len(parts) > 1:
        
        return '/'.join(parts[:6])+'/'+x+'/'+i
    return None

# Configuración del WebDriver
chrome_options = Options()
# chrome_options.add_argument("--headless")  # Ejecuta el navegador en segundo plano
chrome_options.add_argument("--no-sandbox")  # Opcional: Soluciona problemas en algunos entornos
chrome_options.add_argument("--disable-dev-shm-usage")  # Opcional: Mejora el rendimiento en algunos entornos

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

# URL de la página web
url = 'https://inmanga.com/ver/manga/Goblin-Slayer-Year-One/1/7d2722f2-ed53-4198-9d48-b98660ae860d'  # Reemplaza con la URL de tu página

# url = 'https://inmanga.com/ver/manga/Dragon-Ball-Z-Full-Color/1/2855add2-1f48-41b5-87b1-fc08cb049456'

# Extrae el nombre del anime desde la URL
anime_name = extract_anime_name(url)

# Limpiar el nombre del anime para que sea válido como nombre de directorio
anime_name_clean = "".join(char for char in anime_name if char.isalnum() or char.isspace()).replace(" ", "_")

# Abre la página web
driver.get(url)

# Usa una espera explícita para asegurarte de que el elemento esté presente
try:
    select_element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="ChapList"]'))
    )

    # Encuentra todas las opciones dentro del <select>
    options = select_element.find_elements(By.TAG_NAME, 'option')

    cod = []
    capitulo = []

    # Extrae el value y el texto de cada opción
    for option in options:
        cod.append(option.get_attribute('value'))
        capitulo.append(option.text)

    capitulos_complt = {"capitulo" : [], "paginas" : []}

    for x, i in zip(capitulo, cod):
        # Abre la página web
        driver.get(extract_segmento_url(url,x,i))  # Reemplaza con la URL de tu página

        # Usa una espera explícita para asegurarte de que el elemento esté presente
        try:
            select_element = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, '//*[@id="PageList"]'))
            )

            # Encuentra todas las opciones dentro del <select>
            options = select_element.find_elements(By.TAG_NAME, 'option')

            pagina = [opt.text for opt in options]

            boton_next = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, '/html/body/div/section/div/div/div[2]/div[1]/div/div/div[5]/div/button[2]'))
            )

            for _ in range(len(pagina) - 1):
                boton_next.click()
                # time.sleep(2)  # Espera para que se cargue la nueva página

            url_paginas = []

            for idx in range(1, len(pagina) + 1):  # Asumiendo que los índices empiezan en 1
                try:
                    # Encuentra el elemento <a> en base al índice
                    src_pag = WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.XPATH, f'/html/body/div/section/div/div/div[2]/div[2]/div[1]/a[{idx}]'))
                    )

                    # Encuentra el elemento <img> dentro del <a>
                    img_element = src_pag.find_element(By.TAG_NAME, 'img')

                    # Obtén el atributo 'src' del <img>
                    src = img_element.get_attribute('src')

                    url_paginas.append(src)
                except Exception as e:
                    print(f"Error al procesar el elemento {idx}: {e}")

            capitulos_complt["capitulo"].append(x)
            capitulos_complt["paginas"].append(url_paginas)
            # capitulos_complt[x] = url_paginas

        except Exception as e:
            print(f"Error al procesar la página {x}/{i}: {e}")
        break

finally:
    driver.quit()

# Crear DataFrame
df = pd.DataFrame(capitulos_complt)

# Agregar columna con el nombre del anime
df.insert(0, 'anime', anime_name)

# Crear carpetas usando os.makedirs
base_path = os.path.join(os.getcwd(), "mangas", anime_name, "dataset")
os.makedirs(base_path, exist_ok=True)

# Ruta completa para el archivo CSV
csv_path = os.path.join(base_path, "capitulos_completos.csv")

# Exportar el DataFrame a CSV
df.to_csv(csv_path, index=False)

print(f"Archivo CSV guardado en: {csv_path}")


Archivo CSV guardado en: d:\todo relacionado a mangas\script_descarga\Goblin Slayer Year One\dataset\capitulos_complt.csv
